In [1]:
import numpy as np
import pandas as pd
import csv
from scipy import stats
import os
import datetime
import statsmodels.api as sm
import requests
import sys, json
import time
import random
import json
import pickle
import re

In [3]:
os.path.join(os.getcwd(), 'Data_Files', 'bourbon_main.csv')

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/avs-compute-allpurp3/code/Users/asun/Whiskeys/Data_Files/bourbon_main.csv'

In [4]:
ls_whisks = ['american_single_malt', 'bourbon', 'ireland_whiskeys', 'japan_whiskeys', 'rye', 'single_malt', 'single_malt_peated']
ls_whisks

['american_single_malt',
 'bourbon',
 'ireland_whiskeys',
 'japan_whiskeys',
 'rye',
 'single_malt',
 'single_malt_peated']

## WHISKEY REVIEWS

In [5]:
df_combined_revs = pd.DataFrame(columns = ['reviewer_name','reviewer_date', 'reviewer_rating', 'reviewer_notes', 'alc_name', 'alc_type'])
df_combined_revs

,reviewer_name,reviewer_date,reviewer_rating,reviewer_notes,alc_name,alc_type


In [20]:
# DATA PREP. READ IN PICKLES + COMBINE INTO CSV

for filename in os.listdir(os.path.join(os.getcwd(), 'Data_Files')):
    if '.pkl' in filename:
        alc_type = filename.replace('_reviews.pkl', '')
        with open(os.path.join(os.getcwd(), 'Data_Files', filename), 'rb') as f:
            dict_reviews = pickle.load(f)
        for key_id in dict_reviews.keys():
            df_reviews = pd.DataFrame(dict_reviews[key_id]) 
            df_reviews.columns = ['reviewer_name','reviewer_date', 'reviewer_rating', 'reviewer_notes']
            df_reviews['alc_name'] = key_id
            df_reviews['alc_type'] = alc_type
            df_combined_revs = pd.concat([df_combined_revs, df_reviews], axis = 0, ignore_index = True)

In [24]:
print(df_combined_revs.shape)
df_combined_revs.head()

(117345, 6)


,reviewer_name,reviewer_date,reviewer_rating,reviewer_notes,alc_name,alc_type
0,elbucko,"Tasted December 16, 2021",3.75,"Tastes like whiskey, maybe some pear? Great on...",STRANAHAN'S COLORADO WHISKEY,american_single_malt
1,gmrocks,"Tasted December 8, 2021",3.75,This one proved quite popular with group of fr...,STRANAHAN'S COLORADO WHISKEY,american_single_malt
2,Mark-Willis,"Tasted November 27, 2021",4.5,Surprise of the flight consisting of itself Gl...,STRANAHAN'S COLORADO WHISKEY,american_single_malt
3,Dan-Cordial,"Tasted November 13, 2021",3.75,Floral notes,STRANAHAN'S COLORADO WHISKEY,american_single_malt
4,MoparRocker74,"Tasted November 11, 2021",3.75,Really good American single malt. Oaky and Cok...,STRANAHAN'S COLORADO WHISKEY,american_single_malt


In [22]:
df_combined_revs.to_csv(os.path.join(os.getcwd(), 'Data_Files', 'whisk_reviews_combined.csv'), index = False)

<br>

## WHISKEY PROFILES

In [41]:
df_combined_profs = pd.DataFrame(columns = ['Whisk_Name', 'URL', 'Num_Revs', 'Brand', 'Cost', 'Age', 'ABV', 'Flavor', 'Profile'])
df_combined_profs

,Whisk_Name,URL,Num_Revs,Brand,Cost,Age,ABV,Flavor,Profile


In [42]:
# DATA PREP. READ IN PICKLES + COMBINE INTO CSV

for filename in os.listdir(os.path.join(os.getcwd(), 'Data_Files')):
    if '_profiles.csv' in filename:
        df_profile = pd.read_csv(os.path.join(os.getcwd(), 'Data_Files', filename))
        df_combined_profs = pd.concat([df_combined_profs, df_profile], axis = 0, ignore_index = True, sort = False).reset_index(drop = True)

In [43]:
df_combined_profs.head()

,Whisk_Name,URL,Num_Revs,Brand,Cost,Age,ABV,Flavor,Profile
0,STRANAHAN'S COLORADO WHISKEY,https://distiller.com/spirits/stranahan-s-colo...,1159,Stranahan's,spirit-cost cost-3,NaN,47.0,OILY,"{""smoky"":30,""peaty"":0,""spicy"":50,""herbal"":0,""o..."
1,BALCONES TEXAS SINGLE MALT,https://distiller.com/spirits/balcones-texas-s...,915,Balcones,spirit-cost cost-4,NAS,52.5,RICH & FULL BODIED,"{""smoky"":5,""peaty"":0,""spicy"":20,""herbal"":0,""oi..."
2,WESTLAND AMERICAN OAK AMERICAN SINGLE MALT,https://distiller.com/spirits/westland-america...,423,Westland,spirit-cost cost-2,NAS,46.0,VANILLA & SWEET,"{""smoky"":40,""peaty"":0,""spicy"":60,""herbal"":60,""..."
3,CORSAIR TRIPLE SMOKE AMERICAN SINGLE MALT,https://distiller.com/spirits/corsair-triple-s...,312,Corsair,spirit-cost cost-2,NAS,40.0,TART,"{""smoky"":40,""peaty"":25,""spicy"":30,""herbal"":40,..."
4,STRANAHAN'S DIAMOND PEAK (OLD RECIPE),https://distiller.com/spirits/stranahan-s-diam...,306,Stranahan's,spirit-cost cost-3,NAS,47.0,RICH & FULL BODIED,"{""smoky"":50,""peaty"":0,""spicy"":60,""herbal"":10,""..."


In [44]:
df_combined_profs.to_csv(os.path.join(os.getcwd(), 'Data_Files', 'df_whisk_profiles_all.csv'), index = False) # TO PREVENT PICKING UP BY THIS SCRIPT IN FUTURE

<br>

## WHISKEY MAINS

In [45]:
df_combined_main = pd.DataFrame(columns = ['Whisk_Name', 'URL', 'Num_Revs', 'Brand', 'Cost', 'Age', 'ABV', 'Flavor', 'Profile'])
df_combined_main

,Whisk_Name,URL,Num_Revs,Brand,Cost,Age,ABV,Flavor,Profile


In [46]:
# DATA PREP. READ IN PICKLES + COMBINE INTO CSV

for filename in os.listdir(os.path.join(os.getcwd(), 'Data_Files')):
    if '_main.csv' in filename:
        df_main = pd.read_csv(os.path.join(os.getcwd(), 'Data_Files', filename))
        df_combined_main = pd.concat([df_combined_main, df_main], axis = 0, ignore_index = True, sort = False).reset_index(drop = True)

In [47]:
df_combined_main.head()

,Whisk_Name,URL,Num_Revs,Brand,Cost,Age,ABV,Flavor,Profile,Whisk_Type,Whisk_Loc,Communal_Rating,Distiller_Score
0,STRANAHAN'S COLORADO WHISKEY,https://distiller.com/spirits/stranahan-s-colo...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Single Malt,"Colorado, USA",3.70,88.0
1,BALCONES TEXAS SINGLE MALT,https://distiller.com/spirits/balcones-texas-s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Single Malt,"Texas, USA",3.99,92.0
2,WESTLAND AMERICAN OAK AMERICAN SINGLE MALT,https://distiller.com/spirits/westland-america...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Single Malt,"Washington , USA",3.74,89.0
3,CORSAIR TRIPLE SMOKE AMERICAN SINGLE MALT,https://distiller.com/spirits/corsair-triple-s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Single Malt,"Tennessee, USA",3.67,83.0
4,STRANAHAN'S DIAMOND PEAK (OLD RECIPE),https://distiller.com/spirits/stranahan-s-diam...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,American Single Malt,"Colorado, USA",3.88,85.0


In [48]:
df_combined_main.to_csv(os.path.join(os.getcwd(), 'Data_Files', 'df_whisk_main_all.csv'), index = False) # TO PREVENT PICKING UP BY THIS SCRIPT IN FUTURE